<h1>A Machine Learning Hybrid Approach for PCOS Detection Using Ovarian Ultrasound Images</h1>

<h2> The cells below visualize the specifications of the virtual system used in this notebook</h2>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

<h3>Installing Kaggle<h3>

In [ ]:
! pip install kaggle

<h3> Mount google drive to save the models and other variables in this notebook <h3>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

<h3> Several operations were carried out below to download the datast from Kaggle </h3>

In [ ]:
# Making a directory in the colab session that would hold my kaggle API

! mkdir ~/.kaggle

In [ ]:
# Copying the kaggle API credentials file to the newly created kaggle directory

!cp /content/drive/MyDrive/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
# Using the kaggle API to access my kaggle account

! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Dowloading the pcos-detection-using-ultrasound-images dataset into the collab session using my kaggle account

! kaggle datasets download -d anaghachoudhari/pcos-detection-using-ultrasound-images

In [ ]:
# Unzipping the pcos-detection-using-ultrasound-images dataset because Kaggle datasets are often in zipped format

! unzip /content/pcos-detection-using-ultrasound-images.zip

<h3> Importing the libraries used in this notebook </h3>

In [ ]:
import os
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from PIL import Image, UnidentifiedImageError
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

<h2>Data Preprocessing</h2>

<h4>Define the data directory</h4>

In [ ]:
# Define the directory containing the training dataset

data_dir = '/content/data/train'

# Define the directory containing the testing dataset

test_dir = '/content/data/test'

<h4>Define the batch size and image size</h4>

In [ ]:
# Define the batch size for training

batch_size = 64

# Define the dimensions for the images

img_height = 224

img_width = 224

<h4>Preprocess the train data into training and validation set using the image_dataset_from_directory method</h4>

In [ ]:
# Load and preprocess the train dataset

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
data_dir,
validation_split = 0.2,
subset = 'training',
seed = 42,
image_size = (img_height, img_width),
batch_size = batch_size
)

In [ ]:
# Load and preprocess the validation dataset

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
data_dir,
validation_split = 0.2,
subset = 'validation',
seed = 42,
image_size = (img_height, img_width),
batch_size = batch_size
)

<h4>Remove corrupted images from the test directory</h4>

+ Upon looking at the images in the test directory for this dataset, it was observed that there was a number of corrupted images and so to fix that a function was built as seen below

In [ ]:
# Function to remove corrupted images from test_dir

def remove_corrupted_images(directory):

  # Start iterating through the specified directory
  for filename in os.listdir(directory):

    # Defining the file path by joining the directory with the filename
    file_path = os.path.join(directory, filename)

    # trying to open and verify the image in file path
    try:
      # Try to open the image
      img = Image.open(file_path)
      img.verify()  # Additional verification

    # If the verification failed remove the image at the file path
    except (UnidentifiedImageError, OSError) as e:
      # If UnidentifiedImageError or OSError occurs, the file is likely corrupted
      print(f'Removing corrupted file: {file_path}')
      os.remove(file_path)

In [ ]:
# Calling the remove_corrupted_images function on the test directory to remove the corrupted images present

remove_corrupted_images('/content/data/test/infected')
remove_corrupted_images('/content/data/test/notinfected')

<h4>Preprocess the test dataset using ImageDataGenerator

In [ ]:
test_generator = ImageDataGenerator(rescale = 1.0 /255.0)    # Rescale the pixel values to range 0 - 1

test_ds = test_generator.flow_from_directory(
    test_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode ='binary',
    shuffle = False
)

<h3>Visualize the Ovarian Ultrasound Images from DatasetA</h3>

<h4>Get the class names from the training dataset</h4>

In [ ]:
# Get the class names

class_names = train_ds.class_names

# Print the class names

print(class_names)

<h4>Display the count of the classes in the PCOS dataset</h4>

In [ ]:
# Create a count plot of the classes in the PCOS dataset

# Extract the labels from train_ds

labels = []

for idx, label in train_ds:
    labels.extend(label.numpy().tolist())

# Convert the labels to a NumPy array
labels_array = np.array(labels)

# Get the unique labels and their counts
unique_labels, label_counts = np.unique(labels_array, return_counts = True)

# Create a count plot using Matplotlib

plt.figure(figsize = (8, 6))

hist, bins, idx = plt.hist(labels_array, bins = np.arange(labels_array.min(), labels_array.max() +2) - 0.5,
                         rwidth = 0.8, alpha = 0.75, color = 'pink', edgecolor = 'k')

plt.xlabel("PCOS Dataset Classes")
plt.ylabel("Count")
plt.xticks(unique_labels, class_names)
plt.title("Count Plot of the Classes in the PCOS Train set", y = 1.05)


# Add the total count for each class on top of each bar

for i, count in enumerate(label_counts):

    plt.text(unique_labels[i], count, str(count), ha = 'center', va = 'bottom')

plt.show()

<h4>Print 16 images with their labels from the training dataset<h4>

In [ ]:
# Create a figure for displaying images and set size to (10, 10)

plt.figure(figsize = (10, 10))

# Iterate over the first batch of images and labels in the training dataset
for images, labels in train_ds.take(1):

  # Loop through each image in the batch
  for i in range(16):

    # Create a subplot to display each image
    ax = plt.subplot(4, 4, i + 1)

    # Display the image
    plt.imshow(images[i].numpy().astype("uint8"))

    # Set the title of the subplot to the corresponding class name
    plt.title(class_names[labels[i]])

    # Turn off axis labels
    plt.axis("off")

# Show the plot with images and class labels
plt.show()

<h3>Data Augmentation</h3>

+ Data augmentaion was applied to the train set to enhance learning
+ The augmentaions applied can be seen below

In [ ]:
# Data augmentation for the train dataset

data_augmentation = tf.keras.Sequential([
tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),  # Randomly flips the images horizontally.
tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),       # Randomly rotates the images by up to 20%
tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),           # Randomly zooms the images by up to 20%
tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)       # Rescale the pixel values to range 0 - 1
])

# Apply data augmentation to the train dataset

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training = True), y))

<h3> Assigning class weights </h3>

+ As seen in the visualizations above, the dataset is not balanced and so to handle the issue of imbalancing, we assign class weights to the classes to ensure that they contribute equally in the training of the models

In [ ]:
# Calculate the class weights

# Getting the class labels in the dataset
labels = [label.numpy() for _, label in train_ds]

# Joining the labels together and converting to a list
labels = np.concatenate(labels).tolist()

# Using the compute_class_weight method from the sklearn module to calculate the class weights
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(labels),
                                        y = labels
                                    )
# Create a dictionary with the class names as keys and corresponding weights
class_weights = dict(zip(np.unique(labels), class_weights))

class_weights

<h3>Normalization</h3>

+ The data was normalized to promote faster convergence and improved generalization

In [ ]:
# Normalization for val dataset

val_normalization = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

# Apply normalization to the test dataset

val_ds = val_ds.map(lambda x, y: (val_normalization(x), y))

<h3> Create a function to plot model history for training and accuracy </h3>

In [ ]:
# Function for plotting the history for training and validation accuracy and loss

def plot_model_history(history):

  # Plot the training accuracy against the validation accuracy

  # Plot the training accuracy
  plt.plot(history.history['accuracy'], label = 'Training Accuracy')

  # Plot the validation accuracy
  plt.plot(history.history['val_accuracy'], label = 'Validation accuracy')

  plt.title('Training and Validation accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()

  # Display the plot
  plt.show()


  # Plot the training loss against the validation loss

  # Plot the training loss
  plt.plot(history.history['loss'], label = 'Training loss')

  #Plot the validation loss
  plt.plot(history.history['val_loss'], label = 'validation loss')


  plt.title('Training and Validation loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()

  # Display the plot
  plt.show()

<h3> Create a function to generate classification report and confusion matrix </h3>

In [ ]:
# Function to generate classification report and confusion matrix

def generate_classification_report_and_confusion_matrix(y_pred):

  # Define the class labels
  class_labels = ['infected', 'notinfected']

  # Define the true or actual labels of the test dataset
  y_true = test_ds.labels

  # Printing the classification report
  print(classification_report(y_true, y_pred, target_names = class_labels, digits = 4))

  # Plotting the confusion matrix
  cnn_cm = confusion_matrix(y_true, y_pred)

  plt.figure(figsize = (10, 8))

  sns.heatmap(cnn_cm, annot = True, fmt = "d", cmap = "Blues", cbar = True, xticklabels = class_labels,
              yticklabels = class_labels)

  plt.title('Confusion Matrix')
  plt.xlabel('Predicted Labels')
  plt.ylabel('True Labels')

  plt.show()

<h2>CNN Model 1</h2>

<h4>Define the CNN model's architecture</h4>

In [ ]:
model = Sequential([

    # The input layer
    layers.Conv2D(16, 3, padding = 'same', activation = 'relu', input_shape = (img_height,img_width, 3)),
    layers.MaxPooling2D(),

    # First hidden layer
    layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),

    # Second hidden layer
    layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),

    # Flattening layer
    layers.Flatten(),

    # Dense layer
    layers.Dense(64, activation = 'relu'),
    layers.Dropout(0.2),


    # Output layer
    layers.Dense(1, activation = 'sigmoid')
])

# Print the model's architecture summary

model.summary()

<h4>Train the CNN model</h4>

In [ ]:
# Compile the model using the defined optimizer, loss, and metrics

model.compile(

optimizer = Adam(1e-5),
loss = 'BinaryCrossentropy',
metrics = ['accuracy']

)

In [ ]:
%%time
# Define the number of epochs for training

epochs = 30

# Train the model using the train dataset and validate using the val dataset

history = model.fit(
train_ds,
validation_data = val_ds,
epochs = epochs,
class_weight = class_weights
)

# Extract accuracy, validation accuracy, loss, and validation loss from the training history

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Define the range of epochs for plotting

epochs_range = range(epochs)

<h4>Plot the model history</h4>

In [ ]:
plot_model_history(history)

<h4>Get the classification report and confusion matrix</h4>

In [ ]:
%%time
# Getting y_pred

y_pred = model.predict(test_ds)

y_pred = np.round(y_pred).flatten()

In [ ]:
generate_classification_report_and_confusion_matrix(y_pred)

In [ ]:
# Save the model

model.save('/content/drive/MyDrive/PCOS_detection_models/CNN_model1.keras')

<h2>CNN Model 2</h2>

In [ ]:
# Clears the background session before training a new model

tf.keras.backend.clear_session()

<h4>Define the CNN model's architecture</h4>

In [ ]:
model = Sequential([

    # The input layer
    layers.Conv2D(16, 3, padding = 'same', activation = 'relu', input_shape = (img_height,img_width, 3)),
    layers.MaxPooling2D(),

    # First hidden layer
    layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),

    # Second hidden layer
    layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),

    # Flattening layer
    layers.Flatten(),

    # Dense layer
    layers.Dense(128, activation = 'relu'),
    layers.Dropout(0.5),


    # Output layer
    layers.Dense(1, activation = 'sigmoid')
])

# Print the model's architecture summary

model.summary()

In [ ]:
# Compile the model using the defined optimizer, loss, and metrics

model.compile(

optimizer = Adam(1e-6),
loss = 'BinaryCrossentropy',
metrics = ['accuracy']

)

In [ ]:
%%time
# Define the number of epochs for training

epochs = 20

# Train the model using the train dataset and validate using the val dataset

history = model.fit(
train_ds,
validation_data = val_ds,
epochs = epochs,
class_weight = class_weights
)

# Extract accuracy, validation accuracy, loss, and validation loss from the training history

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Define the range of epochs for plotting

epochs_range = range(epochs)

<h4>Plot the model history</h4>

In [ ]:
plot_model_history(history)

<h4>Get the classification report and confusion matrix</h4>

In [ ]:
%%time
# Getting y_pred

y_pred = model.predict(test_ds)

y_pred = np.round(y_pred).flatten()

In [ ]:
generate_classification_report_and_confusion_matrix(y_pred)

In [ ]:
# Save the model

model.save('/content/drive/MyDrive/PCOS_detection_models/CNN_model2.keras')

<h2>CNN Model 3</h2>

In [ ]:
# Clears the background session before training a new model

tf.keras.backend.clear_session()

<h4>Define the CNN model's architecture</h4>

In [ ]:
model = Sequential([

    # The input layer
    layers.Conv2D(16, 3, padding = 'same', activation = 'relu', input_shape = (img_height,img_width, 3)),
    layers.MaxPooling2D(),

    # Hidden layer
    layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),

    # Flattening layer
    layers.Flatten(),

    # Dense layer
    layers.Dense(64, activation = 'relu'),
    layers.Dropout(0.2),

    # Output layer
    layers.Dense(1, activation = 'sigmoid')
])

# Print the model's architecture summary

model.summary()

In [ ]:
# Compile the model using the defined optimizer, loss, and metrics

model.compile(

optimizer = Adam(1e-5),
loss = 'BinaryCrossentropy',
metrics = ['accuracy']

)

In [ ]:
%%time
# Define the number of epochs for training

epochs = 30

# Train the model using the train dataset and validate using the val dataset

history = model.fit(
train_ds,
validation_data = val_ds,
epochs = epochs,
class_weight = class_weights
)

# Extract accuracy, validation accuracy, loss, and validation loss from the training history

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Define the range of epochs for plotting

epochs_range = range(epochs)

<h4>Plot the model history</h4>

In [ ]:
plot_model_history(history)

<h4>Get the classification report and confusion matrix</h4>

In [ ]:
%%time
# Getting y_pred

y_pred = model.predict(test_ds)

y_pred = np.round(y_pred).flatten()

In [ ]:
generate_classification_report_and_confusion_matrix(y_pred)

In [ ]:
# Save the model

model.save('/content/drive/MyDrive/PCOS_detection_models/CNN_model3.keras')

<h2>CNN Model 4</h2>

In [ ]:
# Clears the background session before training a new model

tf.keras.backend.clear_session()

<h4>Define the CNN model's architecture</h4>

In [ ]:
model = Sequential([

    # The input layer
    layers.Conv2D(32, 3, padding = 'same', activation = 'relu', input_shape = (img_height,img_width, 3)),
    layers.MaxPooling2D(),

    # First hidden layer
    layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),

    # Second hidden layer
    layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),

    # Third hidden layer
    layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),

    # Flattening layer
    layers.Flatten(),

    # Dense layer
    layers.Dense(128, activation = 'relu'),
    layers.Dropout(0.2),

    # Output layer
    layers.Dense(1, activation = 'sigmoid')
])

# Print the model's architecture summary

model.summary()

In [ ]:
# Compile the model using the defined optimizer, loss, and metrics

model.compile(

optimizer = Adam(1e-6),
loss = 'BinaryCrossentropy',
metrics = ['accuracy']

)

In [ ]:
%%time
# Define the number of epochs for training

epochs = 20

# Train the model using the train dataset and validate using the val dataset

history = model.fit(
train_ds,
validation_data = val_ds,
epochs = epochs,
class_weight = class_weights
)

# Extract accuracy, validation accuracy, loss, and validation loss from the training history

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Define the range of epochs for plotting

epochs_range = range(epochs)

<h4>Plot the model history</h4>

In [ ]:
plot_model_history(history)

<h4>Get the classification report and confusion matrix</h4>

In [ ]:
%%time
# Getting y_pred

y_pred = model.predict(test_ds)

y_pred = np.round(y_pred).flatten()

In [ ]:
generate_classification_report_and_confusion_matrix(y_pred)

In [ ]:
# Save the model

model.save('/content/drive/MyDrive/PCOS_detection_models/CNN_model4.keras')

<h2>CNN Model 5 - PCONet (Hosain et al, 2022))</h2>

In [ ]:
# Clears the background session before training a new model

tf.keras.backend.clear_session()

<h4>Define the CNN model's architecture</h4>

In [ ]:
model = Sequential([

    # The input layer
    layers.Conv2D(32, 3, padding = 'same', activation = 'relu', input_shape = (img_height,img_width, 3)),
    layers.MaxPooling2D(),

    # First hidden layer
    layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),

    # Second hidden layer
    layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),

    # Second hidden layer
    layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),

    # Third hidden layer
    layers.Conv2D(128, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),

    # Flattening layer
    layers.Flatten(),

    # First Dense layer
    layers.Dense(128, activation = 'relu'),
    layers.Dropout(0.5),

    # Second Dense layer
    layers.Dense(256, activation = 'relu'),
    layers.Dropout(0.5),

    # Output layer
    layers.Dense(1, activation = 'sigmoid')
])

# Print the model's architecture summary

model.summary()

In [ ]:
# Compile the model using the defined optimizer, loss, and metrics

model.compile(

optimizer = Adam(1e-5),
loss = 'BinaryCrossentropy',
metrics = ['accuracy']

)

In [ ]:
%%time
# Define the number of epochs for training

epochs = 50

# Train the model using the train dataset and validate using the val dataset

history = model.fit(
train_ds,
validation_data = val_ds,
epochs = epochs,
class_weight = class_weights
)

# Extract accuracy, validation accuracy, loss, and validation loss from the training history

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Define the range of epochs for plotting

epochs_range = range(epochs)

<h4>Plot the model history</h4>

In [ ]:
plot_model_history(history)

<h4>Get the classification report and confusion matrix</h4>

In [ ]:
%%time
# Getting y_pred

y_pred = model.predict(test_ds)

y_pred = np.round(y_pred).flatten()

In [ ]:
generate_classification_report_and_confusion_matrix(y_pred)

In [ ]:
# Save the model

model.save('/content/drive/MyDrive/PCOS_detection_models/CNN_model5.keras')

<h2>Transfer Learning Models</h2>

<h4>Define a class transfer_learning_models to build all the transfer learning models in this notebook<h4>

There are several methods in this class:
+ Method to initializes the class transfer_learning_models
+ Method to freeze the layers of the pre-trained model for initial training
+ Method to unfreeze the layers and compile the pre-trained model to finetune the model
+ Method to define the architecture of the pretrained model
+ Method to fit the pretrained model on the PCOS dataset
+ Method to fit the finetuned pretrained model on the PCOS dataset
+ Method to plot the training and validation accuracy and loss for the pretrained model
+ Method to print classification report and plot confusion matrix
+ Method to save the models


In [ ]:
# Define a class transfer_learning_models

class transfer_learning_models:

    # Method to initializes the class transfer_learning_models
    def __init__(self, base_model):

        # Clears the background session before training a new model
        tf.keras.backend.clear_session()

        # Loads a pre-trained model
        self.base_model = base_model(weights = 'imagenet', include_top = False,
                                           input_shape = (224, 224, 3))
        self.model = None
        self.history = None
        self.finetuned_history = None
        self.model_name = None

    # Method to freeze the layers of the pre-trained model
    def freeze_pretrained_model_layers(self):

        self.base_model.trainable = False

    # Method to unfreeze the layers and compile the pre-trained model
    def unfreeze_pretrained_model_layers(self, learning_rate):

        self.base_model.trainable = True

        self.model.summary()

        self.model.compile(

            optimizer = Adam(learning_rate),
            loss = 'BinaryCrossentropy',
            metrics = ['accuracy']
        )

    # Method to define the architecture of the pretrained model
    def define_pretrained_model_architecture(self, dropout_value, learning_rate):

        inputs = tf.keras.Input(shape = (224, 224, 3))
        x = inputs
        x = self.base_model(x, training = False)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dropout(dropout_value)(x)

        outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

        self.model = tf.keras.Model(inputs, outputs)

        self.model.compile(

            optimizer = Adam(learning_rate),
            loss = 'BinaryCrossentropy',
            metrics = ['accuracy']
        )

        return self.model.summary()


    # Method to fit the pretrained model on the PCOS dataset
    def fit_model(self, epochs):
        self.history = self.model.fit(

            train_ds,
            epochs = epochs,
            class_weight = class_weights,
            validation_data = val_ds
        )

    # Method to fit the finetuned pretrained model on the PCOS dataset
    def fit_fine_tuned_model(self, epochs):
        self.fine_tuned_history = self.model.fit(
            train_ds,
            epochs = epochs,
            class_weight = class_weights,
            validation_data = val_ds,
        )

    # Method to plot the training and validation accuracy and loss for the pretrained model
    def plot_accuracy_and_loss(self, model_name, acc_y_lower_lim, loss_y_upper_limit):

        initial_epochs = self.history.epoch[-1]

        acc = self.history.history['accuracy'] + self.fine_tuned_history.history['accuracy']
        val_acc = self.history.history['val_accuracy'] + self.fine_tuned_history.history['val_accuracy']
        loss = self.history.history['loss'] + self.fine_tuned_history.history['loss']
        val_loss = self.history.history['val_loss'] + self.fine_tuned_history.history['val_loss']

        # Calculate the y-axis tick positions for increments of 0.2
        acc_y_ticks = np.arange(acc_y_lower_lim, 1.02, 0.02)
        loss_y_ticks = np.arange(0, loss_y_upper_limit + 0.02, 0.02)



        plt.figure(figsize = (8, 8))
        plt.subplot(2, 1, 1)
        plt.plot(acc, label = 'Training Accuracy')
        plt.plot(val_acc, label = 'Validation Accuracy')

        plt.ylim([acc_y_lower_lim, 1])
        plt.plot([initial_epochs - 0.15, initial_epochs - 0.15],
        plt.ylim(), label = 'Start Fine Tuning')
        plt.legend(loc = 'lower right')
        plt.title(f'Training and Validation Accuracy for {model_name}')
        plt.subplot(2, 1, 2)
        plt.plot(loss, label = 'Training Loss')
        plt.plot(val_loss, label = 'Validation Loss')


        plt.ylim([0, loss_y_upper_limit])
        plt.plot([initial_epochs - 0.15,initial_epochs - 0.15],
        plt.ylim(), label = 'Start Fine Tuning')
        plt.legend(loc = 'upper right')
        plt.title(f'Training and Validation Loss for {model_name}')
        plt.xlabel('epoch')

        plt.show()

    # Method to print classification report and plot confusion matrix
    def classification_report_and_confusion_metrics(self, test_ds):
      y_true = test_ds.labels

      class_labels = ['infected', 'notinfected']

      y_pred = self.model.predict(test_ds)

      y_pred = np.round(y_pred).flatten()

      print(classification_report(y_true, y_pred, target_names = class_labels, digits = 4))


      cnn_cm = confusion_matrix(y_true, y_pred)

      plt.figure(figsize=(10, 8))

      sns.heatmap(cnn_cm, annot = True, fmt = "d", cmap = "Blues", cbar = True, xticklabels = class_labels,
                  yticklabels = class_labels)

      plt.title('CNN Model Confusion Matrix')
      plt.xlabel('Predicted Labels')
      plt.ylabel('True Labels')

      plt.show()

    # Method to save the models
    def save_model(self, path_to_save_model):

      self.model.save(path_to_save_model)

<h2>InceptionV3 Model</h2>

In [ ]:
# Creates an instance of the class transfer_learning_models for InceptionV3

InceptionV3 = transfer_learning_models(tf.keras.applications.InceptionV3)

In [ ]:
# Freeze all the layers of the InceptionV3 model

InceptionV3.freeze_pretrained_model_layers()

In [ ]:
# Define the architecture and print the summary for the InceptionV3 model

InceptionV3.define_pretrained_model_architecture(0.2, 1e-4)

In [ ]:
%%time
# Fit the InceptionV3 model on the PCOS dataset

InceptionV3.fit_model(10)

In [ ]:
# Finetune the InceptionV3 model

InceptionV3.unfreeze_pretrained_model_layers(1e-7)

In [ ]:
%%time
# Fit the finetuned InceptionV3 model on the PCOS dataset

InceptionV3.fit_fine_tuned_model(10)

In [ ]:
# Plot the training and validation accuracy and loss

InceptionV3.plot_accuracy_and_loss('InceptionV3', 0.50, 1.0)

In [ ]:
%%time
# Print the classification report and plot the confusion matrix

InceptionV3.classification_report_and_confusion_metrics(test_ds)

In [ ]:
# Save the InceptionV3 model

InceptionV3.save_model('/content/drive/MyDrive/PCOS_detection_models/InceptionV3.keras')

<h2>MobileNetV2 Model</h2>

In [ ]:
# Creates an instance of the class transfer_learning_models for MobileNetV2

MobileNetV2 = transfer_learning_models(tf.keras.applications.MobileNetV2)

In [ ]:
# Freeze all the layers of the MobileNetV2 model

MobileNetV2.freeze_pretrained_model_layers()

In [ ]:
# Define the architecture and print the summary for the MobileNetV2  model

MobileNetV2.define_pretrained_model_architecture(0.2, 1e-4)

In [ ]:
%%time
# Fit the MobileNetV2 model on the PCOS dataset

MobileNetV2.fit_model(10)

In [ ]:
# Finetune the MobileNetV2 model

MobileNetV2.unfreeze_pretrained_model_layers(1e-8)

In [ ]:
%%time
# Fit the finetuned MobileNetV2 model on the PCOS dataset

MobileNetV2.fit_fine_tuned_model(10)

In [ ]:
# Plot the training and validation accuracy and loss

MobileNetV2.plot_accuracy_and_loss('MobileNetV2', 0.50, 1.0)

In [ ]:
%%time
# Print the classification report and plot the confusion matrix

MobileNetV2.classification_report_and_confusion_metrics(test_ds)

In [ ]:
# Save the MobileNetV2 model

MobileNetV2.save_model('/content/drive/MyDrive/PCOS_detection_models/MobileNetV2.keras')

<h2>VGG16 Model</h2>

In [ ]:
# Creates an instance of the class transfer_learning_models for VGG16

VGG16 = transfer_learning_models(tf.keras.applications.VGG16)

In [ ]:
# Freeze all the layers of the VGG16 model

VGG16.freeze_pretrained_model_layers()

In [ ]:
# Define the architecture and print the summary for the VGG16 model

VGG16.define_pretrained_model_architecture(0.2, 1e-4)

In [ ]:
%%time
# Fit the VGG16 model on the PCOS dataset

VGG16.fit_model(10)

In [ ]:
# Finetune the VGG16 model

VGG16.unfreeze_pretrained_model_layers(1e-7)

In [ ]:
%%time
# Fit the finetuned VGG16 model on the PCOS dataset

VGG16.fit_fine_tuned_model(20)

In [ ]:
# Plot the training and validation accuracy and loss

VGG16.plot_accuracy_and_loss('VGG16', 0.50, 1.0)

In [ ]:
%%time
# Print the classification report and plot the confusion matrix

VGG16.classification_report_and_confusion_metrics(test_ds)

In [ ]:
# Save the VGG16 model

VGG16.save_model('/content/drive/MyDrive/PCOS_detection_models/VGG16.keras')

<h2>ResNet50 Model</h2>

In [ ]:
# Creates an instance of the class transfer_learning_models for ResNet50

ResNet50 = transfer_learning_models(tf.keras.applications.ResNet50)

In [ ]:
# Freeze all the layers of the ResNet50 model

ResNet50.freeze_pretrained_model_layers()

In [ ]:
# Define the architecture and print the summary for the ResNet50  model

ResNet50.define_pretrained_model_architecture(0.5, 1e-4)

In [ ]:
%%time
# Fit the ResNet50 model on the PCOS dataset

ResNet50.fit_model(10)

In [ ]:
# Finetune the ResNet50 model

ResNet50.unfreeze_pretrained_model_layers(1e-7)

In [ ]:
%%time
# Fit the finetuned ResNet50 model on the PCOS dataset

ResNet50.fit_fine_tuned_model(50)

In [ ]:
# Plot the training and validation accuracy and loss

ResNet50.plot_accuracy_and_loss('ResNet50', 0.50, 1.0)

In [ ]:
%%time
# Print the classification report and plot the confusion matrix

ResNet50.classification_report_and_confusion_metrics(test_ds)

In [ ]:
# Save the ResNet50 model

ResNet50.save_model('/content/drive/MyDrive/PCOS_detection_models/ResNet 50.keras')